In [1]:
import numpy as np
import pandas as pd
from statistics import mean, median, variance, stdev
from jubatus.common import Datum
from jubatus.classifier.client import Classifier
import glob
from pprint import pprint

JUBATUS_SERVER = "192.168.11.26"

In [2]:
"""
# Jubatus config file

{
  "method" : "NN", # nearest neighbor
  "parameter" : {
    "method" : "euclid_lsh",
    "parameter" : {
      "hash_num" : 512
    },
    "local_sensitivity" : 0.1,
    "nearest_neighbor_num" : 5
  },
  "converter" : {
    "num_filter_types" : {},
    "num_filter_rules" : [],
    "num_types" : {},
    "num_rules" : [
      { "key" : "*", "type" : "num"}
    ]
  }
}
"""

'\n# Jubatus config file\n\n{\n  "method" : "NN", # nearest neighbor\n  "parameter" : {\n    "method" : "euclid_lsh",\n    "parameter" : {\n      "hash_num" : 512\n    },\n    "local_sensitivity" : 0.1,\n    "nearest_neighbor_num" : 5\n  },\n  "converter" : {\n    "num_filter_types" : {},\n    "num_filter_rules" : [],\n    "num_types" : {},\n    "num_rules" : [\n      { "key" : "*", "type" : "num"}\n    ]\n  }\n}\n'

In [3]:
# ここに学習用データを配置する
path = './CsvData4Train/*'
files = []
files = glob.glob(path)
print(files)

['./CsvData4Train/nagama_downup.csv', './CsvData4Train/oshima_rightleft.csv', './CsvData4Train/nagama_rightleft.csv', './CsvData4Train/nagama_leftclock.csv', './CsvData4Train/oishi_updown.csv', './CsvData4Train/oshima_leftclock.csv', './CsvData4Train/nagama_updown.csv', './CsvData4Train/oishi_downup.csv', './CsvData4Train/oishi_rightclock.csv', './CsvData4Train/oshima_leftright.csv', './CsvData4Train/nagama_leftright.csv', './CsvData4Train/nagama_rightclock.csv', './CsvData4Train/oshima_rightclock.csv', './CsvData4Train/oishi_rightleft.csv', './CsvData4Train/oshima_updown.csv', './CsvData4Train/oishi_leftclock.csv', './CsvData4Train/oshima_downup.csv', './CsvData4Train/oishi_leftright.csv', './CsvData4Train/SampleData.csv']


In [4]:
columns = ('ax_mean_0','ay_mean_0','az_mean_0','mx_mean_0','my_mean_0','mz_mean_0',
           'ax_mean_1','ay_mean_1','az_mean_1','mx_mean_1','my_mean_1','mz_mean_1',
           'ax_mean_2','ay_mean_2','az_mean_2','mx_mean_2','my_mean_2','mz_mean_2',
           'ax_mean_3','ay_mean_3','az_mean_3','mx_mean_3','my_mean_3','mz_mean_3',
           'ax_mean_4','ay_mean_4','az_mean_4','mx_mean_4','my_mean_4','mz_mean_4',
           'ax_mean_5','ay_mean_5','az_mean_5','mx_mean_5','my_mean_5','mz_mean_5',
           'ax_mean_6','ay_mean_6','az_mean_6','mx_mean_6','my_mean_6','mz_mean_6',
           'ax_mean_7','ay_mean_7','az_mean_7','mx_mean_7','my_mean_7','mz_mean_7',
           'ax_mean_8','ay_mean_8','az_mean_8','mx_mean_8','my_mean_8','mz_mean_8',
           'ax_mean_9','ay_mean_9','az_mean_9','mx_mean_9','my_mean_9','mz_mean_9',)

# columns = ('ax_mean','ay_mean','az_mean',
#            'mx_mean','my_mean','mz_mean',)

# columns = ('ax_mean','ay_mean','az_mean',
#            'mx_mean','my_mean','mz_mean',
#            'ax_var','ay_var','az_var',
#            'mx_var','my_var','mz_var',)

In [5]:
nagama_downup     = pd.read_csv('./CsvData4Train/nagama_downup.csv', names=columns)
nagama_updown     = pd.read_csv('./CsvData4Train/nagama_updown.csv', names=columns)
nagama_leftright  = pd.read_csv('./CsvData4Train/nagama_leftright.csv', names=columns)
nagama_rightleft  = pd.read_csv('./CsvData4Train/nagama_rightleft.csv', names=columns)
nagama_leftclock  = pd.read_csv('./CsvData4Train/nagama_leftclock.csv', names=columns)
nagama_rightclock = pd.read_csv('./CsvData4Train/nagama_rightclock.csv', names=columns)
oishi_downup      = pd.read_csv('./CsvData4Train/oishi_downup.csv', names=columns)
oishi_updown      = pd.read_csv('./CsvData4Train/oishi_updown.csv', names=columns)
oishi_leftright   = pd.read_csv('./CsvData4Train/oishi_leftright.csv', names=columns)
oishi_rightleft   = pd.read_csv('./CsvData4Train/oishi_rightleft.csv', names=columns)
oishi_leftclock   = pd.read_csv('./CsvData4Train/oishi_leftclock.csv', names=columns)
oishi_rightclock  = pd.read_csv('./CsvData4Train/oishi_rightclock.csv', names=columns)
oshima_downup     = pd.read_csv('./CsvData4Train/oshima_downup.csv', names=columns)
oshima_updown     = pd.read_csv('./CsvData4Train/oshima_updown.csv', names=columns)
oshima_leftright  = pd.read_csv('./CsvData4Train/oshima_leftright.csv', names=columns)
oshima_rightleft  = pd.read_csv('./CsvData4Train/oshima_rightleft.csv', names=columns)
oshima_leftclock  = pd.read_csv('./CsvData4Train/oshima_leftclock.csv', names=columns)
oshima_rightclock = pd.read_csv('./CsvData4Train/oshima_rightclock.csv', names=columns)

#label_1 = pd.read_csv('./CsvData4Train/label_1.csv', names=columns)
#label_2 = pd.read_csv('./CsvData4Train/label_2.csv', names=columns)
# ... こんな感じでラベルに対応するものを入れていけばOK

In [6]:
def train(df, label = "label"):
    labels = [label for i in range(len(df))]
    # preparing for jubatus
    features = df.as_matrix()
    columns = df.columns.tolist()
    
    train_data = [] # train data should be Datum type
    for x, y in zip(features, labels):
        d = Datum({key: value for key, value in zip(columns, x)})
        train_data.append([y, d])
    client = Classifier(JUBATUS_SERVER, 9199, '')
    client.train(train_data)

In [7]:
## テスト用データを用意する
def create_test_data(df):
    columns = df.columns.tolist()
    features = df.as_matrix()
    test_data = []
    for feature in features:
        d = Datum({key: value for key, value in zip(columns, feature)})
        test_data.append(d)
    return test_data

def get_most_likely(result):
    return max(result, key = lambda x: x.score)

In [8]:
dataset = [nagama_downup,nagama_updown,nagama_leftright,nagama_rightleft,nagama_leftclock,nagama_rightclock,
           oishi_downup,oishi_updown,oishi_leftright,oishi_rightleft,oishi_leftclock,oishi_rightclock,
           oshima_downup,oshima_updown,oshima_leftright,oshima_rightleft,oshima_leftclock,oshima_rightclock,]

In [9]:
client = Classifier(JUBATUS_SERVER, 9199, '')
client.clear() # 既に学習済みのモデルがある場合はクリア

train(nagama_downup[:15]    , label='downup')
train(nagama_updown[:15]    , label='updown')
train(nagama_leftright[:15] , label='leftright')
train(nagama_rightleft[:15] , label='rightleft')
train(nagama_leftclock[:15] , label='leftclock')
train(nagama_rightclock[:15], label='rightclock')
train(oishi_downup[:15]     , label='downup')
train(oishi_updown[:15]     , label='updown')
train(oishi_leftright[:15]  , label='leftright')
train(oishi_rightleft[:15]  , label='rightleft')
train(oishi_leftclock[:15]  , label='leftclock')
train(oishi_rightclock[:15] , label='rightclock')
train(oshima_downup[:15]    , label='downup')
train(oshima_updown[:15]    , label='updown')
train(oshima_leftright[:15] , label='leftright')
train(oshima_rightleft[:15] , label='rightleft')
train(oshima_leftclock[:15] , label='leftclock')
train(oshima_rightclock[:15], label='rightclock')

# train(label_1_train, label="label_1")
# train(label_2_train, label="label_2")

In [10]:
var_name = lambda val : [k for k, v in globals().items() if id(v) == id(val)]

In [11]:
for dset in dataset:
    test_data = create_test_data(dset[15:])
    print(var_name(dset)[0])
    for testdata in [test_data]:
        client = Classifier(JUBATUS_SERVER, 9199, '')
        results = client.classify(testdata)
        for result in results:
            print(get_most_likely(result))
        #print("done")

nagama_downup
estimate_result{label: downup, score: 9.898183929657119}
estimate_result{label: downup, score: 9.876431849375553}
estimate_result{label: downup, score: 9.895878157215318}
estimate_result{label: downup, score: 9.868482062833595}
estimate_result{label: downup, score: 9.879264696436627}
nagama_updown
estimate_result{label: updown, score: 9.936535562198346}
estimate_result{label: updown, score: 9.923358786767087}
estimate_result{label: updown, score: 9.919470295523055}
estimate_result{label: updown, score: 9.900894487124212}
estimate_result{label: updown, score: 9.92394045030827}
estimate_result{label: updown, score: 9.915213176084153}
estimate_result{label: updown, score: 9.920053965176056}
nagama_leftright
estimate_result{label: leftright, score: 9.91967220479406}
estimate_result{label: leftright, score: 9.917348657727443}
estimate_result{label: leftright, score: 9.928589473238972}
estimate_result{label: leftright, score: 9.89742276909361}
estimate_result{label: leftright, 

In [12]:
var_name = lambda val : [k for k, v in globals().items() if id(v) == id(val)]

In [13]:
from jubatus.common import Datum
from jubatus.classifier.client import Classifier

columns = ('ax_mean_0','ay_mean_0','az_mean_0','mx_mean_0','my_mean_0','mz_mean_0',
           'ax_mean_1','ay_mean_1','az_mean_1','mx_mean_1','my_mean_1','mz_mean_1',
           'ax_mean_2','ay_mean_2','az_mean_2','mx_mean_2','my_mean_2','mz_mean_2',
           'ax_mean_3','ay_mean_3','az_mean_3','mx_mean_3','my_mean_3','mz_mean_3',
           'ax_mean_4','ay_mean_4','az_mean_4','mx_mean_4','my_mean_4','mz_mean_4',
           'ax_mean_5','ay_mean_5','az_mean_5','mx_mean_5','my_mean_5','mz_mean_5',
           'ax_mean_6','ay_mean_6','az_mean_6','mx_mean_6','my_mean_6','mz_mean_6',
           'ax_mean_7','ay_mean_7','az_mean_7','mx_mean_7','my_mean_7','mz_mean_7',
           'ax_mean_8','ay_mean_8','az_mean_8','mx_mean_8','my_mean_8','mz_mean_8',
           'ax_mean_9','ay_mean_9','az_mean_9','mx_mean_9','my_mean_9','mz_mean_9',)

def get_most_likely(result):
    return max(result, key = lambda x: x.score)

def classify(columns, values, srvr_ip, srvr_port):
    d = Datum({key: value for key, value in zip(columns, values)})
    client = Classifier(srvr_ip, srvr_port, '')
    results = client.classify([d])
    return get_most_likely(results[0])

# JUBATUS_SERVER = "192.168.3.15"
sample_data = [-0.729666667,-0.126555556,1.497333333,-27.35388889,128.0281111,-156.3126667,-2,-0.496888889,1.687555556,162.329,-71.88155556,25.86377778,-1.551,-1.14125,0.03675,241.021875,-98.569,244.289125,-0.13375,-0.725375,-1.013375,7.162125,55.396,249.992,1.20025,-0.79825,0.033,-239.531625,97.423375,196.454875,1.43575,-0.676125,1.45475,-179.01225,201.07625,-136.342125,-0.087875,-1.3905,0.885875,-15.658375,249.992,-250,-1.903375,-1.6055,1.245625,186.110375,157.194,-227.281625,-2,-1.693375,0.824,249.992,47.37,67.744375,-0.914625,-1.5655,-0.017375,236.479375,-42.942875,249.891875]
classify(columns, sample_data, JUBATUS_SERVER, 9199)

estimate_result{label: leftclock, score: 9.837968191398838}